In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import os
import json
import regex as re
import json

In [21]:
# Load raw ingredient data
with open('data/‏‏parsed_ingredients_dict_revised.json', 'r', encoding ='utf8') as json_file:
    extracted_ingredients_carbs_vegan = json.load(json_file)

### Parse and filter dictionary entries

In [8]:
import re
import json

parsed_ingredient_dict = {}

for ingredient, raw in extracted_ingredients_carbs_vegan.items():
    # If it’s already a dict, use it directly
    if isinstance(raw, dict):
        parsed_ingredient_dict[ingredient] = raw
        continue

In [9]:
# Convert to a DataFrame for further processing
rows = []
for ingredient, info in parsed_ingredient_dict.items():
    rows.append({
        "ingredient_name": ingredient,
        "is_keto": info.get("carbs_per_100g"),
        "is_vegan": info.get("is_vegan")
    })

In [10]:
# Create the DataFrame
ingredient_embed_df = pd.DataFrame(rows, columns=["ingredient_name", "is_keto", "is_vegan"])

In [12]:
# check for missing values
ingredient_embed_df.isna().sum()

ingredient_name    0
is_keto            1
is_vegan           1
dtype: int64

In [13]:
# Drop rows with missing keto or vegan values
ingredient_embed_df.dropna(inplace=True)

# Standardize text: lowercase ingredient names
ingredient_embed_df['ingredient_name'] = ingredient_embed_df['ingredient_name'].apply(lambda x: x.lower())

# Convert numeric carb values to boolean keto flag
ingredient_embed_df['is_keto'] = ingredient_embed_df['is_keto'].apply(lambda x: x <= 10)

In [15]:
len(ingredient_embed_df)

3967

### Create Embeddings for each ingredient

In [18]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
ingredient_names = ingredient_embed_df["ingredient_name"].tolist()

# Compute embeddings
embeddings = model.encode(ingredient_names, convert_to_numpy=True)

# Store embeddings
ingredient_embed_df["embedding"] = list(embeddings)

In [20]:
# Save final df to parquet
ingredient_embed_df.to_parquet("data/embed_ingredients_df.parquet", index=False)

In [29]:
with open('data/parsed_ingredients_carbs_vegan.json', 'r', encoding ='utf8') as json_file:
    parsed_extracted_ingredients_carbs_vegan = json.load(json_file)